In [1]:
# Google Colab compatibility
project_folder = ''

In [2]:
from custompackage.load_data import *
from custompackage.load_architecture import *
from custompackage.traintestloop import *
import numpy as np

In [3]:
class_set = 0

use_cuda = torch.cuda.is_available()

# Initialize settings
bs = 256
weighting = 'paired'
trials = 10
epochs = 2000
trees_set = [1]

# Load class-dataset list
# classes = np.load(project_folder + 'results/classes.npy', allow_pickle=True)

classes = [[3, 5, 'mnist'],
           [0, 6, 'fmnist'],
           [2, 6, 'kmnist'],
           [14, 17, 'emnist']]

# Initialize final test loss and accuracy variables
loss = np.zeros((len(classes), trials, len(trees_set)))
acc = np.zeros((len(classes), trials, len(trees_set)))

# For each dataset enumerated from classes list
for j, (t1, t2, ds) in enumerate(classes):
    print(t1, t2, ds)
    # Load data loaders
    trainloaders, validloaders, testloader = dataset_weighted_split_all(bs, t1, t2, weighting, trials, ds, permute=False)
    # Initialize input size for model initialization purposes
    input_size = trainloaders[0].dataset.tensors[0][0].shape[0]
    # For each trial
    for k, trees in enumerate(trees_set):
        # For every k-tree defined by trees_set
        for i in range(trials):
            print(j, i, k)
            # Initialize the ktree model
            if use_cuda:
                model = ktree_gen(ds=ds, Repeats=trees, Padded=True).cuda()
            else:
                model = ktree_gen(ds=ds, Repeats=trees, Padded=True)

            #Train and test ktree, assigning loss and acc values
            loss_curve, acc_curve, loss[j,i,k], acc[j,i,k], model_t = train_test_ktree(model, trainloaders[i],
                                                                                  validloaders[i], testloader, epochs = epochs, randorder=False, use_cuda=use_cuda)
            # Save accuracy and loss arrays
            np.save(project_folder + 'results/ktree_acc_orig_'+str(class_set)+'.npy', acc)
            np.save(project_folder + 'results/ktree_loss_orig_'+str(class_set)+'.npy', loss)

3 5 mnist
0 0 0
EarlyStopping counter: 1 out of 60
EarlyStopping counter: 2 out of 60
EarlyStopping counter: 3 out of 60
EarlyStopping counter: 4 out of 60
EarlyStopping counter: 5 out of 60
EarlyStopping counter: 6 out of 60
EarlyStopping counter: 7 out of 60
EarlyStopping counter: 8 out of 60
EarlyStopping counter: 9 out of 60
EarlyStopping counter: 10 out of 60
EarlyStopping counter: 11 out of 60
EarlyStopping counter: 12 out of 60
EarlyStopping counter: 13 out of 60
EarlyStopping counter: 14 out of 60
EarlyStopping counter: 15 out of 60
EarlyStopping counter: 16 out of 60
EarlyStopping counter: 17 out of 60
EarlyStopping counter: 18 out of 60
EarlyStopping counter: 19 out of 60
EarlyStopping counter: 20 out of 60
EarlyStopping counter: 21 out of 60
EarlyStopping counter: 22 out of 60
EarlyStopping counter: 23 out of 60
EarlyStopping counter: 24 out of 60
EarlyStopping counter: 25 out of 60
EarlyStopping counter: 26 out of 60
EarlyStopping counter: 27 out of 60
EarlyStopping counter

KeyboardInterrupt: 

In [ ]:
class_set = 0
acc = np.load(project_folder + 'results/ktree_acc_orig_'+str(class_set)+'.npy', allow_pickle=True)
print("RESULTS:")
for j, (t1, t2, ds) in enumerate(classes):
    print(f"Dataset: {ds} / Pair: {t1}-{t2}")
    for k, trees in enumerate(trees_set):
        print(f"{trees}-tree")
        print(f"Accuracy: mean = {round(np.mean(acc[j,:,k]), 4)}, standard deviation = {round(np.std(acc[j,:,k]), 4)}")